In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/babble/data/')

os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_spouse'

from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

In [3]:
candidates = session.query(Spouse).filter(Spouse.split == 1).all()[:100]
print(len(candidates))

100


In [4]:
import os
from mturk_processing import MTurkHelper
helper = MTurkHelper(candidates, num_hits=25)

In [6]:
output_csv_path= os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/mturk_spouse_01_out.csv'
explanations = helper.postprocess(output_csv_path, candidates=candidates, verbose=False)

Num HITs unique: 25
Num HITs total: 75


Exception: Candidate linking failed.

In [7]:
exp_iterator = iter(explanations)

NameError: name 'explanations' is not defined

In [8]:
exp = exp_iterator.next()
from snorkel.viewer import SentenceNgramViewer
print(exp.condition)
sv = SentenceNgramViewer([exp.candidate], session, n_per_page=3, height=150)
sv

NameError: name 'exp_iterator' is not defined

In [ ]:
for exp in explanations[:10]: print(exp.condition)

In [ ]:
from snorkel.contrib.babble import Explanation
exp = Explanation("person1 equals person2", True, name='test_exp')
homemades = [exp]

In [ ]:
from snorkel.contrib.babble import Babbler
user_lists = {}
babbler = Babbler(Spouse, explanations, user_lists)
# babbler.apply(parallelism=1)

In [ ]:
lfs = babbler.generate_lfs()

In [ ]:
babbler.filter_duplicate_semantics()

In [ ]:
babbler.filter_consistency()

In [ ]:
from snorkel.contrib.babble import sem_to_str

for parse in babbler.parses:
    print(sem_to_str(parse.semantics))

In [ ]:
babbler.get_explanations()

In [ ]:
%time babbler.generate_label_matrix(split=1, parallelism=1)

In [ ]:
# babbler.load_matrix(session, split=1)

In [ ]:
babbler.filter_uniform_signatures()

In [ ]:
babbler.filter_duplicate_signatures()

In [ ]:
for p in babbler.parses: print(sem_to_str(p.semantics))

In [ ]:
babbler.get_explanations()

In [ ]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

In [ ]:
babbler.label_matrix.lf_stats(session, labels=L_gold_dev)

In [ ]:
# candidates = session.query(Spouse).filter(Spouse.split == 1).all()
# print(len(candidates))

In [ ]:
lf = babbler.lfs[-1]
yes = 0
no = 0
for c in candidates:
    if lf(c):
        yes += 1
    else:
        no += 1
print(yes)
print(no)

In [ ]:
def f(c):
    return 1 if (
        c[0].get_span().strip() in [p.text for p in get_sentence_phrases(c[0])] and
        c[1].get_span().strip() in [p.text for p in get_sentence_phrases(c[0])]) else 0

for c in candidates:
    if lf(c) != f(c):
        print(c[0].get_span(), c[1].get_span())
        pprint([p.text for p in get_sentence_phrases(c[0])])
        import pdb; pdb.set_trace()

In [ ]:
from pprint import pprint
from snorkel.contrib.babble import *
yes = 0
no = 0
for c in candidates:
    if (c[0].get_span().strip() in [p.text for p in get_sentence_phrases(c[0])] and 
        c[1].get_span().strip() in [p.text for p in get_sentence_phrases(c[0])]):
        yes += 1
    else:
        no += 1
print(yes)
print(no)

In [ ]:
import numpy as np
print(np.sum(abs(babbler.label_matrix), 0))